<a href="https://colab.research.google.com/github/albertaillet/KTH-EQ2425/blob/master/project3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb --quiet
!wandb login

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="project3", entity="eq2425_2022p3_aillet_bonato")

In [ ]:
wandb.config = {
    "learning_rate": 1e-3,
    "epochs": 300,
    "batch_size": 64,
    "dropout": False,
    "batch_normalization": False,
    "data_shuffling": False,
}

In [ ]:
model = models.Sequential([
    layers.Conv2D(
        kernel_size=(5, 5),
        strides=1,
        padding="valid",
        filters=24,
        activation='relu',
    ),
    layers.MaxPooling2D(
        pool_size=(2, 2), 
        strides=2, 
    ),
    layers.Conv2D(
        kernel_size=(3, 3),
        strides=1,
        padding="valid",
        filters=48,
        activation='relu',
    ),
    layers.MaxPooling2D(
        pool_size=(2, 2), 
        strides=2, 
    ),
    layers.Conv2D(
        kernel_size=(3, 3),
        strides=1,
        padding="valid",
        filters=96,
    ),
    layers.MaxPooling2D(
        pool_size=(2, 2), 
        strides=2, 
    ),
    layers.Dense(
        units=512,
        activation="relu"
    ),
    layers.Dense(
        units=10,
        activation="softmax"
    ),
])